In [1]:
from dotenv import load_dotenv

assert load_dotenv("../../.env")

In [2]:
import ragna

ragna.__version__

'0.1.dev21+g7f8ec2d.d20230925071852'

In [3]:
from ragna import demo_config, Config

demo_config

Config(state_database_url='sqlite://', queue_database_url='redis://127.0.0.1:6379', ragna_api_url='http://127.0.0.1:31476', ragna_ui_url='http://127.0.0.1:31477', local_cache_root=PosixPath('/home/philip/.cache/ragna'), document_class=<class 'ragna.core.LocalDocument'>, upload_token_secret='ef1ffa68d4d397af5373c2bf2095abef937be87b8a52e4debabae0c0dfcf9f38', upload_token_ttl=30, registered_source_storage_classes={'Ragna/DemoSourceStorage': <class 'ragna.source_storage._demo.RagnaDemoSourceStorage'>}, registered_assistant_classes={'Ragna/DemoAssistant': <class 'ragna.assistant._demo.RagnaDemoAssistant'>})

In [4]:
document_path = "demo_document.txt"

with open(document_path, "w") as file:
    file.write("Ragna is an open-source RAG orchestration app\n")

In [5]:
from ragna.core import Rag
from ragna.assistant import (
    RagnaDemoAssistant,
    OpenaiGpt35Turbo16kAssistant,
    OpenaiGpt4Assistant,
)
from ragna.source_storage import ChromaSourceStorage, RagnaDemoSourceStorage

rag = Rag(demo_config)

async with await rag.new_chat(
    documents=[document_path],
    source_storage=RagnaDemoSourceStorage,
    assistant=RagnaDemoAssistant,
) as chat:
    prompt = "What is Ragna?"
    answer = await chat.answer(prompt)

print(answer)

RagnaException: Worker process terminated unexpectedly

In [ ]:
import itertools
import asyncio
from pprint import pprint

source_storages = [ChromaSourceStorage]
assistants = [OpenaiGpt35Turbo16kAssistant, OpenaiGpt4Assistant]


async def answer_prompt(source_storage, assistant):
    chat = await rag.new_chat(
        documents=[document_path],
        source_storage=source_storage,
        assistant=assistant,
    )
    await chat.start()
    return await chat.answer(prompt)


experiments = {
    (source_storage.display_name(), assistant.display_name()): answer_prompt(
        source_storage, assistant
    )
    for source_storage, assistant in itertools.product(source_storages, assistants)
}

pprint(experiments)

In [ ]:
results = dict(zip(experiments.keys(), await asyncio.gather(*experiments.values())))
pprint(results)